In [1]:
# main.py
# 2021-06-09
# Scrapes the AIS data for Innovation and writes it to a text file
# References mapping.py, which takes scraped data and writes to a map
# Scraping code from https://digital-geography.com/vessel-tracking-the-python-way/



# region Import Modules

import urllib.request
from bs4 import BeautifulSoup
import re
import folium
import pandas as pd
import numpy as np
from datetime import datetime
import locale
#locale.setlocale(locale.LC_ALL, 'en_US') # as we need to deal with names of months later on.
import os
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from arcgis.geometry import lengths, areas_and_lengths, project
from arcgis.geometry import Point, Polyline, Polygon, Geometry
from arcgis.mapping import create_symbol
import pandas as pd

# endregion


ModuleNotFoundError: No module named 'bs4'

In [2]:
# region Tracked Vessels
IMOS = [9603453, 9383754, 9275153, 9483413, 9282144] # Set up IMO numbers for vessel tracking
IMOS_Names = ['Innovation', 'Olympic Triton - LDTVO', 'Boa Sub C', 'Furore-G', 'Wind of Pride'] # Identify tracked vessels

#IMOS
#IMOS_Names

# endregion


['Innovation', 'Boa Sub C', 'Furore-G']

In [3]:
# region AIS data scrape data from vesselfinder.com
hdr = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive'}
items = []
for IMO in IMOS:
    url = r'https://www.vesselfinder.com/en/vessels/VOS-TRAVELLER-IMO-' + str(IMO)
    req = urllib.request.Request(url, None, hdr)
    with urllib.request.urlopen(req) as response:
        the_page = response.read()
    parsed_html = BeautifulSoup(the_page)
    a = parsed_html.p
    tables = parsed_html.findAll("table")
    for table in tables:
        if table.findParent("table") is None:
            for row in table.findAll('tr'):
                aux = row.findAll('td')

                try:
                    if aux[0].string == "Vessel Name":
                        name = aux[1].string
                    if aux[0].string == "Position received":
                        #print(aux[1].get("data-title"))
                        #zeit = datetime.strptime(aux[1].get("data-title"), '%b %d, %Y %H:%M %Z')
                        #print(zeit)
                except:
                    print("strange table found")
    a = parsed_html.p
    [b,c, d] = a.text.partition('nates ')
    [coords,f,g] = d.partition(')')
    coordsSplit = coords.split(" / ")


    def dms2dd(degrees, direction):
        dd = float(degrees);
        if direction == 'S' or direction == 'W':
            dd *= -1
        return dd


    def parse_dms(dms):
        parts = re.split(' ', dms)
        lat = dms2dd(parts[0], parts[1])
        return lat

    
    lat = parse_dms(coordsSplit[0])
    lng = parse_dms(coordsSplit[1])
    items.append((lat, lng, name, zeit))
# endregion

Jun 11, 2021 00:20 UTC
2021-06-11 00:20:00
Jun 11, 2021 00:20 UTC
2021-06-11 00:20:00
strange table found
Jun 11, 2021 00:10 UTC
2021-06-11 00:10:00
strange table found


In [4]:
#items


[(47.17, -2.57592, 'INNOVATION', datetime.datetime(2021, 6, 11, 0, 20)),
 (47.17342, -2.56905, 'BOA SUB C', datetime.datetime(2021, 6, 11, 0, 20)),
 (47.18493, -2.53788, 'FURORE-G', datetime.datetime(2021, 6, 11, 0, 10))]

In [5]:
# region Write positions to .txt file
filename = 'ship_positions.txt'
if os.path.exists(filename):
    append_write = 'a' # append if already exists
    fw = open(filename,append_write)
else:
    append_write = 'w' # make a new file if not
    fw = open(filename,append_write)
    fw.write("lat;lng;name;time\n")
for item in items:
    fw.write("%3.5f;%3.5f;%s;%s\n" % item)
fw.close()
# endregion

In [64]:
# region Set-up area Folium
Area1 = [(47.20222972,47.1826325,47.14691889,47.12426444,47.11305028,47.14761917,47.17143083,47.1826325,47.16279444,47.17749861, 47.20899361), (-2.691135278,-2.686666667,-2.699031944,-2.579898889,-2.565479167,-2.497501389,-2.5187875,-2.569599167,-2.579898889,-2.635173889,-2.660236389)]
df2 = pd.DataFrame.from_records(np.transpose(Area1))
#df2

# endregion

,0,1
0,47.202230,-2.691135
1,47.182632,-2.686667
2,47.146919,-2.699032
3,47.124264,-2.579899
4,47.113050,-2.565479
5,47.147619,-2.497501
6,47.171431,-2.518788
7,47.182632,-2.569599
8,47.162794,-2.579899
9,47.177499,-2.635174


In [61]:

# region Set-up area Arcgis
Area = [(-2.691135278,-2.686666667,-2.699031944,-2.579898889,-2.565479167,-2.497501389,-2.5187875,-2.569599167,-2.579898889,-2.635173889,-2.660236389),(47.20222972,47.1826325,47.14691889,47.12426444,47.11305028,47.14761917,47.17143083,47.1826325,47.16279444,47.17749861, 47.20899361)]
np.transpose(Area)
df1 = pd.DataFrame.from_records(np.transpose(Area))
#df1
df1 = df1.iloc[::-1].reset_index(drop=True)
#df1
# endregion

,0,1
0,-2.660236,47.208994
1,-2.635174,47.177499
2,-2.579899,47.162794
3,-2.569599,47.182632
4,-2.518788,47.171431
5,-2.497501,47.147619
6,-2.565479,47.113050
7,-2.579899,47.124264
8,-2.699032,47.146919
9,-2.686667,47.182632


In [62]:
# region Plot onto a map
from arcgis.gis import GIS
gis = GIS()
map = gis.map()
df = pd.DataFrame.from_records(items)
df.columns = ['y', 'x', 'name', 'zeit']
ships = gis.content.import_data(df)

geom = Geometry({'spatialReference': {'latestWkid': 4326,
                'wkid':4326}, 
                'rings': [df1[:].values.tolist()]
                   })


map.add_layer(ships)

map.center = [lat, lng]
map.center = {'x': -2.6, 'y': 47, 
               "spatialReference" : {"wkid" : 4326}}
map.zoom = 10
map.basemap = 'satellite'
symbol = create_symbol(geometry_type='polygon', cmap='prism')
#map.draw(geom, symbol=symbol)
#map

# endregion

MapView(layout=Layout(height='400px', width='100%'))

In [66]:
# region Folium map options

m = folium.Map(location= [47.2, -2.6], 
               zoom_start=9.5,
              tiles='CartoDB positron')
tooltip = "Innovation"

folium.Marker([df.y[0], df.x[0]],
              popup="<i>Innovation</i>", 
              tooltip=tooltip
             ).add_to(m)
folium.Marker([df.y[1], df.x[1]],
              popup="<i>LDTVO</i>", 
              tooltip="Olympic Triton"
             ).add_to(m)
folium.Marker([df.y[4], df.x[4]],
              popup="<i>LDTVO</i>", 
              tooltip="Wind of Pride"
             ).add_to(m)
folium.vector_layers.Polygon(locations = [df2[:].values.tolist()],
              popup="<i>SNA</i>",
               tooltip = "SNA"
              ).add_to(m)
m
# endregion